# 1. Import and Install Dependencies

In [4]:
# !pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

In [5]:
import cv2 as cv
import numpy as np
import copy
import os
from matplotlib import pyplot as plt
import time
from collections import deque
import itertools
import mediapipe as mp
from model import KeyPointClassifier
from model import PointHistoryClassifier

# 2. Keypoints using MP Holistic

In [6]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
mp_hands = mp.solutions.hands

In [7]:
def mediapipe_detection(image, model):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv.cvtColor(image, cv.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [8]:
# def draw_styled_landmarks(image, results):
#     # # Draw face connections
#     # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
#     #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
#     #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
#     #                          ) 
#     # # Draw pose connections
#     # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
#     #                          mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
#     #                          mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
#     #                          ) 
#     # Draw left hand connections
#     mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
#                              mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
#                              mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
#                              ) 
#     # Draw right hand connections  
#     mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
#                              mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
#                              mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
#                              ) 


def draw_landmarks(image, landmark_point):
    if len(landmark_point) > 0:
        # Thumb
        cv.line(image, tuple(landmark_point[2]), tuple(landmark_point[3]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[2]), tuple(landmark_point[3]),
                (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[3]), tuple(landmark_point[4]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[3]), tuple(landmark_point[4]),
                (255, 255, 255), 2)

        # Index finger
        cv.line(image, tuple(landmark_point[5]), tuple(landmark_point[6]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[5]), tuple(landmark_point[6]),
                (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[6]), tuple(landmark_point[7]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[6]), tuple(landmark_point[7]),
                (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[7]), tuple(landmark_point[8]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[7]), tuple(landmark_point[8]),
                (255, 255, 255), 2)

        # Middle finger
        cv.line(image, tuple(landmark_point[9]), tuple(landmark_point[10]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[9]), tuple(landmark_point[10]),
                (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[10]), tuple(landmark_point[11]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[10]), tuple(landmark_point[11]),
                (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[11]), tuple(landmark_point[12]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[11]), tuple(landmark_point[12]),
                (255, 255, 255), 2)

        # Ring finger
        cv.line(image, tuple(landmark_point[13]), tuple(landmark_point[14]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[13]), tuple(landmark_point[14]),
                (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[14]), tuple(landmark_point[15]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[14]), tuple(landmark_point[15]),
                (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[15]), tuple(landmark_point[16]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[15]), tuple(landmark_point[16]),
                (255, 255, 255), 2)

        # Little finger
        cv.line(image, tuple(landmark_point[17]), tuple(landmark_point[18]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[17]), tuple(landmark_point[18]),
                (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[18]), tuple(landmark_point[19]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[18]), tuple(landmark_point[19]),
                (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[19]), tuple(landmark_point[20]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[19]), tuple(landmark_point[20]),
                (255, 255, 255), 2)

        # Palm
        cv.line(image, tuple(landmark_point[0]), tuple(landmark_point[1]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[0]), tuple(landmark_point[1]),
                (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[1]), tuple(landmark_point[2]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[1]), tuple(landmark_point[2]),
                (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[2]), tuple(landmark_point[5]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[2]), tuple(landmark_point[5]),
                (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[5]), tuple(landmark_point[9]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[5]), tuple(landmark_point[9]),
                (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[9]), tuple(landmark_point[13]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[9]), tuple(landmark_point[13]),
                (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[13]), tuple(landmark_point[17]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[13]), tuple(landmark_point[17]),
                (255, 255, 255), 2)
        cv.line(image, tuple(landmark_point[17]), tuple(landmark_point[0]),
                (0, 0, 0), 6)
        cv.line(image, tuple(landmark_point[17]), tuple(landmark_point[0]),
                (255, 255, 255), 2)

    # Key Points
    for index, landmark in enumerate(landmark_point):
        if index == 0:  # 手首1
            cv.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 1:  # 手首2
            cv.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 2:  # 親指：付け根
            cv.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 3:  # 親指：第1関節
            cv.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 4:  # 親指：指先
            cv.circle(image, (landmark[0], landmark[1]), 8, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 8, (0, 0, 0), 1)
        if index == 5:  # 人差指：付け根
            cv.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 6:  # 人差指：第2関節
            cv.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 7:  # 人差指：第1関節
            cv.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 8:  # 人差指：指先
            cv.circle(image, (landmark[0], landmark[1]), 8, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 8, (0, 0, 0), 1)
        if index == 9:  # 中指：付け根
            cv.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 10:  # 中指：第2関節
            cv.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 11:  # 中指：第1関節
            cv.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 12:  # 中指：指先
            cv.circle(image, (landmark[0], landmark[1]), 8, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 8, (0, 0, 0), 1)
        if index == 13:  # 薬指：付け根
            cv.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 14:  # 薬指：第2関節
            cv.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 15:  # 薬指：第1関節
            cv.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 16:  # 薬指：指先
            cv.circle(image, (landmark[0], landmark[1]), 8, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 8, (0, 0, 0), 1)
        if index == 17:  # 小指：付け根
            cv.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 18:  # 小指：第2関節
            cv.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 19:  # 小指：第1関節
            cv.circle(image, (landmark[0], landmark[1]), 5, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 5, (0, 0, 0), 1)
        if index == 20:  # 小指：指先
            cv.circle(image, (landmark[0], landmark[1]), 8, (255, 255, 255),
                      -1)
            cv.circle(image, (landmark[0], landmark[1]), 8, (0, 0, 0), 1)

    return image


def draw_bounding_rect(use_brect, image, brect):
    if use_brect:
        # Outer rectangle
        cv.rectangle(image, (brect[0], brect[1]), (brect[2], brect[3]),
                     (0, 0, 0), 1)

    return image


def draw_info_text(image, brect, handedness, hand_sign_text,
                   finger_gesture_text):
    cv.rectangle(image, (brect[0], brect[1]), (brect[2], brect[1] - 22),
                 (0, 0, 0), -1)

    info_text = handedness.classification[0].label[0:]
    if hand_sign_text != "":
        info_text = info_text + ':' + hand_sign_text
    cv.putText(image, info_text, (brect[0] + 5, brect[1] - 4),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv.LINE_AA)

#     if finger_gesture_text != "":
#         cv.putText(image, "Finger Gesture:" + finger_gesture_text, (10, 60),
#                    cv.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 4, cv.LINE_AA)
#         cv.putText(image, "Finger Gesture:" + finger_gesture_text, (10, 60),
#                    cv.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2,
#                    cv.LINE_AA)

    return image


def draw_point_history(image, point_history):
    for index, point in enumerate(point_history):
        if point[0] != 0 and point[1] != 0:
            cv.circle(image, (point[0], point[1]), 1 + int(index / 2),
                      (152, 251, 152), 2)

    return image


def draw_info(image, fps, mode, number):
    cv.putText(image, "FPS:" + str(fps), (10, 30), cv.FONT_HERSHEY_SIMPLEX,
               1.0, (0, 0, 0), 4, cv.LINE_AA)
    cv.putText(image, "FPS:" + str(fps), (10, 30), cv.FONT_HERSHEY_SIMPLEX,
               1.0, (255, 255, 255), 2, cv.LINE_AA)

    mode_string = ['Logging Key Point', 'Logging Point History','Drawing Mode']
    if 1 <= mode <= 2:
        cv.putText(image, "MODE:" + mode_string[mode - 1], (10, 90),
                   cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1,
                   cv.LINE_AA)
        if 0 <= number <= 9:
            cv.putText(image, "NUM:" + str(number), (10, 110),
                       cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1,
                       cv.LINE_AA)
    return image



In [162]:

def calc_bounding_rect(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]

        landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x, y, w, h = cv.boundingRect(landmark_array)

    return [x, y, x + w, y + h]


def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point


def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list


def pre_process_point_history(image, point_history):
    image_width, image_height = image.shape[1], image.shape[0]

    temp_point_history = copy.deepcopy(point_history)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, point in enumerate(temp_point_history):
        if index == 0:
            base_x, base_y = point[0], point[1]

        temp_point_history[index][0] = (temp_point_history[index][0] -
                                        base_x) / image_width
        temp_point_history[index][1] = (temp_point_history[index][1] -
                                        base_y) / image_height

    # Convert to a one-dimensional list
    temp_point_history = list(
        itertools.chain.from_iterable(temp_point_history))

    # def mul(i):
    #     return i*100000

    # temp_point_history=list(map(mul,temp_point_history))

    return temp_point_history


In [215]:
def calc_landmark_list3D(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y,landmark.z])

    return landmark_point

In [10]:
keypoint_classifier = KeyPointClassifier()

point_history_classifier = PointHistoryClassifier()

# Coordinate history #################################################################
history_length = 20
point_history = deque(maxlen=history_length)


In [220]:

cap = cv.VideoCapture(0)
# Set mediapipe model 
with mp_hands.Hands(static_image_mode=False,max_num_hands=1,min_detection_confidence=0.7, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():

        # Read feed
        ret, image = cap.read()
        image = cv.flip(image, 1)  # Mirror display
        debug_image = copy.deepcopy(image)

        # Make detections
        image, results = mediapipe_detection(image, hands)
        # print(results)
        
        if results.multi_hand_landmarks is not None:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks,
                                                  results.multi_handedness):
                # Bounding box calculation
                brect = calc_bounding_rect(debug_image, hand_landmarks)
                # Landmark calculation
                landmark_list = calc_landmark_list(debug_image, hand_landmarks)
                # print(landmark_list)
                # Conversion to relative coordinates / normalized coordinates
                pre_processed_landmark_list = pre_process_landmark(
                    landmark_list)
                pre_processed_point_history_list = pre_process_point_history(
                    debug_image, point_history)
                 
                hand_sign_id = keypoint_classifier(pre_processed_landmark_list)

                if hand_sign_id == 2:  # Point gesture
                    point_history.append(landmark_list[8])
                else:
                    point_history.append([0, 0])

                # Drawing part
                debug_image = draw_bounding_rect(True, debug_image, brect)
                debug_image = draw_landmarks(debug_image, landmark_list)
                debug_image = draw_info_text(
                    debug_image,
                    brect,
                    handedness,
                    "NA",
                    "NA"
                    # point_history_classifier_labels[most_common_fg_id[0][0]]
                )
                landmark_list_3d=calc_landmark_list3D(debug_image,hand_landmarks)
                print(landmark_list_3d)
        else:
            point_history.append([0,0])
        
        debug_image = draw_point_history(debug_image, point_history)
        
        # Screen reflection #############################################################
        cv.imshow('Hand Gesture Recognition', debug_image)
        # Show to screen
        # cv.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv.waitKey(10) & 0xFF == 27:
            break
    cap.release()
    cv.destroyAllWindows()

[[218, 441, 5.781799359283468e-07], [258, 420, -0.03803304210305214], [291, 386, -0.05864369124174118], [311, 354, -0.07573729753494263], [325, 326, -0.09471717476844788], [254, 333, -0.05985599756240845], [267, 280, -0.09551378339529037], [273, 244, -0.12034779042005539], [277, 215, -0.13692402839660645], [221, 332, -0.06138875335454941], [215, 274, -0.09738345444202423], [213, 235, -0.12463093549013138], [212, 202, -0.14290019869804382], [194, 344, -0.06497234106063843], [182, 295, -0.09820044785737991], [175, 261, -0.116546131670475], [171, 232, -0.12735338509082794], [172, 364, -0.07119809091091156], [157, 328, -0.10130561888217926], [148, 304, -0.11287359148263931], [140, 282, -0.11787578463554382]]
[[222, 414, 8.4152054569131e-07], [267, 384, -0.038724277168512344], [300, 342, -0.05909433960914612], [321, 303, -0.07695218920707703], [341, 271, -0.09621613472700119], [257, 287, -0.04053769260644913], [265, 239, -0.07269203662872314], [272, 205, -0.09963742643594742], [277, 175, -0

# 3. Extract Keypoint Values

In [216]:
def extract_keypoints(landmark_list3d):
    temp_landmark_list = copy.deepcopy(landmark_list3d)
    temp_landmark_list = list(itertools.chain.from_iterable(temp_landmark_list))
    return temp_landmark_list

In [65]:
def extract_keypoint_history(landmark_list,base_landmark_list=None):
    temp_landmark_list = copy.deepcopy(landmark_list)

    for index,landmark_point in enumerate(temp_landmark_list):
        temp_landmark_list[index][0]-=base_landmark_list[index][0]
        temp_landmark_list[index][1]-=base_landmark_list[index][1]
    temp_landmark_list = list(itertools.chain.from_iterable(temp_landmark_list))

    # Convert to a one-dimensional list
    temp_landmark_list = list(itertools.chain.from_iterable(temp_landmark_list))

    return temp_landmark_list


In [172]:
result_test = extract_keypoints(pre_processed_landmark_list)
result_test


[0.0,
 0.0,
 3.134077700922148e-09,
 0.16216216216216217,
 -0.04633204633204633,
 -0.00019456884863293768,
 0.2972972972972973,
 -0.15444015444015444,
 -0.00030288287226297683,
 0.39768339768339767,
 -0.2625482625482625,
 -0.00039075789649514156,
 0.4864864864864865,
 -0.3552123552123552,
 -0.0004846459995365511,
 0.18146718146718147,
 -0.46332046332046334,
 -0.0002309941330701688,
 0.25868725868725867,
 -0.6640926640926641,
 -0.0003745167814626657,
 0.3088803088803089,
 -0.8108108108108109,
 -0.0004887198274200027,
 0.3474903474903475,
 -0.9227799227799228,
 -0.0005666922533374035,
 0.04633204633204633,
 -0.49034749034749037,
 -0.0002219263737496262,
 0.08108108108108109,
 -0.7142857142857143,
 -0.000349279708843894,
 0.10424710424710425,
 -0.8725868725868726,
 -0.00045795235279444104,
 0.1274131274131274,
 -1.0,
 -0.0005341927632401809,
 -0.08108108108108109,
 -0.4594594594594595,
 -0.00023000854207742168,
 -0.0888030888030888,
 -0.6718146718146718,
 -0.00035006642111480005,
 -0.0810

# 4. Setup Folders for Collection

In [286]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Dataset') 

# Actions that we try to detect
actions = np.array(['s', 'z','l'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 50 frames in length
sequence_length = 50

# Folder start
start_folder = 31

In [251]:
for action in actions: 
    # dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(start_folder,start_folder+no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
            print("Dataset created")
        except:
            print("Dataset already exists")
            pass

Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset created
Dataset 

# 5. Collect Keypoint Values for Training and Testing

In [252]:
cap = cv.VideoCapture(0)
# Set mediapipe model 
with mp_hands.Hands(static_image_mode=False,max_num_hands=1,min_detection_confidence=0.7, min_tracking_confidence=0.5) as hands:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):
                # print(frame_num)
                # Read feed
                ret, image = cap.read()
                image = cv.flip(image, 1)  # Mirror display
                debug_image = copy.deepcopy(image)

                # Make detections
                image, results = mediapipe_detection(image, hands)

                if results.multi_hand_landmarks is not None:
                    for hand_landmarks, handedness in zip(results.multi_hand_landmarks,
                                                        results.multi_handedness):
                        # Bounding box calculation
                        brect = calc_bounding_rect(debug_image, hand_landmarks)
                        # Landmark calculation
                        landmark_list = calc_landmark_list(debug_image, hand_landmarks)
                        # Conversion to relative coordinates / normalized coordinates
                        pre_processed_landmark_list = pre_process_landmark(
                            landmark_list)
                        pre_processed_point_history_list = pre_process_point_history(
                            debug_image, point_history)
                        
                        hand_sign_id = keypoint_classifier(pre_processed_landmark_list)

                        if hand_sign_id == 2:  # Point gesture
                            point_history.append(landmark_list[8])
                        else:
                            point_history.append([0, 0])

                        # Drawing part
                        debug_image = draw_bounding_rect(True, debug_image, brect)
                        debug_image = draw_landmarks(debug_image, landmark_list)
                        debug_image = draw_info_text(
                            debug_image,
                            brect,
                            handedness,
                            "NA",
                            "NA"
                            # point_history_classifier_labels[most_common_fg_id[0][0]]
                        )

                        landmark_list_3d=calc_landmark_list3D(debug_image,hand_landmarks)
                
                else:
                    point_history.append([0,0])

                debug_image = draw_point_history(debug_image, point_history)
                    
                # NEW Apply wait logic
                # print(frame_num)

                cv.putText(debug_image, f"Frame number:{frame_num}", (15,40), 
                            cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv.LINE_AA)
                if frame_num == 0: 
                    cv.putText(debug_image, 'STARTING COLLECTION', (120,200), 
                            cv.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv.LINE_AA)
                    cv.putText(debug_image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                            cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv.LINE_AA)
                    
                    # Show to screen
                    cv.imshow('OpenCV Feed', debug_image)
                    cv.waitKey(500)
                    
                    #Collecting keypoints
                    # base_landmark_list=copy.deepcopy(landmark_list)
                    # keypoints = extract_keypoint_history(landmark_list,base_landmark_list)
                    # print(base_landmark_list)
                    # print(landmark_list)
                    # print(keypoints)
                else: 
                    cv.putText(debug_image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                            cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv.LINE_AA)
                    
                    # Show to screen
                    cv.imshow('OpenCV Feed', debug_image)

                    # keypoints=extract_keypoints(landmark_list)
                    # keypoints=extract_keypoint_history(landmark_list,base_landmark_list)
                    # print(base_landmark_list)
                    # print(landmark_list)
                    # print(keypoints)
    
                # print(keypoints)
                # cv.imshow('OpenCV Feed', debug_image)
                
                keypoints=extract_keypoints(landmark_list_3d)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                # Break gracefully
                if cv.waitKey(10) & 0xFF == 27:
                    break
                    
    cap.release()
    cv.destroyAllWindows()

In [173]:
cap.release()
cv.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [253]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [287]:
label_map = {label:num for num, label in enumerate(actions)}

In [288]:
label_map

{'s': 0, 'z': 1, 'l': 2}

In [289]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [290]:
np.array(sequences).shape

(180, 50, 63)

In [291]:
np.array(labels).shape

(180,)

In [292]:
X = np.array(sequences)

In [293]:
X.shape

(180, 50, 63)

In [294]:
y = to_categorical(labels).astype(int)

In [295]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [296]:
y_test.shape

(9, 3)

# 7. Build and Train LSTM Neural Network

In [267]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard,EarlyStopping

In [302]:
log_dir = os.path.join('TrainingLOGS/Logs_3dmodel2')
tb_callback = TensorBoard(log_dir=log_dir)
# Callback for early stopping
# es_callback = EarlyStopping(monitor='catagorical_aaccuracy',mode='max',patience=20, verbose=1)

In [299]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(50,63)))
# model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [304]:
model2=Sequential()
model2.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(50,63)))
model2.add(LSTM(128, return_sequences=False, activation='relu'))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(actions.shape[0], activation='softmax'))

model2.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model2.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])



Epoch 1/2000
6/6 [==============================] - 4s 261ms/step - loss: 751.8466 - categorical_accuracy: 0.3099
Epoch 2/2000
6/6 [==============================] - 2s 277ms/step - loss: 1232.4961 - categorical_accuracy: 0.2807
Epoch 3/2000
6/6 [==============================] - 2s 278ms/step - loss: 1492.5507 - categorical_accuracy: 0.2632
Epoch 4/2000
6/6 [==============================] - 2s 278ms/step - loss: 998.3357 - categorical_accuracy: 0.3392
Epoch 5/2000
6/6 [==============================] - 2s 326ms/step - loss: 686.8590 - categorical_accuracy: 0.3743
Epoch 6/2000
6/6 [==============================] - 2s 283ms/step - loss: 1014.1052 - categorical_accuracy: 0.3392
Epoch 7/2000
6/6 [==============================] - 1s 237ms/step - loss: 572.9517 - categorical_accuracy: 0.3567
Epoch 8/2000
6/6 [==============================] - 2s 257ms/step - loss: 640.9550 - categorical_accuracy: 0.3860
Epoch 9/2000
6/6 [==============================] - 2s 311ms/step - loss: 1136.1748 -

KeyboardInterrupt: 

In [300]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [301]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
6/6 [==============================] - 6s 480ms/step - loss: 780.9524 - categorical_accuracy: 0.3041
Epoch 2/2000
6/6 [==============================] - 3s 424ms/step - loss: 791.7045 - categorical_accuracy: 0.4094
Epoch 3/2000
6/6 [==============================] - 3s 497ms/step - loss: 1476.7070 - categorical_accuracy: 0.3918
Epoch 4/2000
6/6 [==============================] - 2s 396ms/step - loss: 1326.8771 - categorical_accuracy: 0.3275
Epoch 5/2000
6/6 [==============================] - 3s 450ms/step - loss: 783.3432 - categorical_accuracy: 0.3567
Epoch 6/2000
6/6 [==============================] - 2s 423ms/step - loss: 1346.0449 - categorical_accuracy: 0.3333
Epoch 7/2000
6/6 [==============================] - 3s 439ms/step - loss: 1664.2313 - categorical_accuracy: 0.3567
Epoch 8/2000
6/6 [==============================] - 3s 444ms/step - loss: 2075.6228 - categorical_accuracy: 0.4211
Epoch 9/2000
6/6 [==============================] - 3s 447ms/step - loss: 4205.7197

KeyboardInterrupt: 

In [271]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_27 (LSTM)              (None, 50, 64)            32768     
                                                                 
 lstm_28 (LSTM)              (None, 50, 128)           98816     
                                                                 
 lstm_29 (LSTM)              (None, 64)                49408     
                                                                 
 dense_26 (Dense)            (None, 64)                4160      
                                                                 
 dense_27 (Dense)            (None, 32)                2080      
                                                                 
 dense_28 (Dense)            (None, 4)                 132       
                                                                 
Total params: 187,364
Trainable params: 187,364
Non-tr

# 8. Make Predictions

In [272]:
res = model.predict(X_test)

1/1 [==============================] - 1s 526ms/step


In [273]:
actions[np.argmax(res[4])]

's'

In [274]:
actions[np.argmax(y_test[4])]

'l'

# 9. Save Weights

In [276]:
model.save('TrainedModels/3d2.h5')

In [298]:
del model

In [277]:
model.load_weights('TrainedModels/3d2.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [206]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [280]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 67ms/step


In [281]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [282]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[9, 0],
        [3, 0]],

       [[0, 9],
        [0, 3]],

       [[9, 0],
        [3, 0]],

       [[9, 0],
        [3, 0]]], dtype=int64)

In [283]:
accuracy_score(ytrue, yhat)

0.25

# 11. Test in Real Time

In [284]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv.putText(output_frame, actions[num], (0, 85+num*40), cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv.LINE_AA)
        
    return output_frame

In [285]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv.VideoCapture(0)
# Set mediapipe model 
with mp_hands.Hands(static_image_mode=False,max_num_hands=1,min_detection_confidence=0.7, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():

        # Read feed
        ret, image = cap.read()
        image = cv.flip(image, 1)  # Mirror display
        debug_image = copy.deepcopy(image)

        # Make detections
        image, results = mediapipe_detection(image, hands)
        # print(results)
        
        if results.multi_hand_landmarks is not None:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks,
                                                  results.multi_handedness):
                # Bounding box calculation
                brect = calc_bounding_rect(debug_image, hand_landmarks)
                # Landmark calculation
                landmark_list = calc_landmark_list(debug_image, hand_landmarks)
                # Conversion to relative coordinates / normalized coordinates
                pre_processed_landmark_list = pre_process_landmark(
                    landmark_list)
                pre_processed_point_history_list = pre_process_point_history(
                    debug_image, point_history)
                
                hand_sign_id = keypoint_classifier(pre_processed_landmark_list)

                if hand_sign_id == 2:  # Point gesture
                    point_history.append(landmark_list[8])
                else:
                    point_history.append([0, 0])

                # Drawing part
                debug_image = draw_bounding_rect(True, debug_image, brect)
                debug_image = draw_landmarks(debug_image, landmark_list)
                debug_image = draw_info_text(
                    debug_image,
                    brect,
                    handedness,
                    "NA",
                    "NA"
                    # point_history_classifier_labels[most_common_fg_id[0][0]]
                )
            landmark_list_3d=calc_landmark_list3D(debug_image,hand_landmarks)

        else:
            point_history.append([0,0])
        
        debug_image = draw_point_history(debug_image, point_history)
        
        # if cv.waitKey(10) & 0xFF==ord('b'):
        #     base_landmark_list=copy.deepcopy(landmark_list)
        #     print("base created",base_landmark_list)

        # 2. Prediction logic
        keypoints = extract_keypoints(landmark_list_3d)

        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            debug_image = prob_viz(res, actions, debug_image, colors)
            
        cv.rectangle(debug_image, (0,0), (640, 40), (245, 117, 16), -1)
        cv.putText(debug_image, ' '.join(sentence), (3,30), 
                       cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
        
        # Show to screen
        cv.imshow('OpenCV Feed', debug_image)

        # Break gracefully
        if cv.waitKey(10) & 0xFF == 27:
            break
    cap.release()
    cv.destroyAllWindows()

1/1 [==============================] - 0s 400ms/step
s
1/1 [==============================] - 0s 33ms/step
s
1/1 [==============================] - 0s 56ms/step
s
1/1 [==============================] - 0s 50ms/step
s
1/1 [==============================] - 0s 67ms/step
s
1/1 [==============================] - 0s 44ms/step
s
1/1 [==============================] - 0s 65ms/step
s
1/1 [==============================] - 0s 50ms/step
s
1/1 [==============================] - 0s 46ms/step
s
1/1 [==============================] - 0s 33ms/step
s
1/1 [==============================] - 0s 54ms/step
s
1/1 [==============================] - 0s 50ms/step
s
1/1 [==============================] - 0s 60ms/step
s
1/1 [==============================] - 0s 49ms/step
s
1/1 [==============================] - 0s 72ms/step
s
1/1 [==============================] - 0s 34ms/step
s
1/1 [==============================] - 0s 47ms/step
s
1/1 [==============================] - 0s 34ms/step
s
1/1 [======================